In [7]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [12]:
X = df_cleaned.iloc[:1000,:]
X

ticker,AA,ABM,ABT,ADI,ADM,ADX,AEE,AEG,AEM,AEP,...,XLI,XLK,XLP,XLU,XLV,XLY,XOM,XRX,YUM,ZTR
03/01/2000,-0.013042,-0.009188,-0.007117,-0.036071,0.000000,-0.001867,-0.007707,0.014739,-0.008658,-0.017734,...,-0.021967,-0.003376,-0.016349,-0.017490,-0.022942,-0.028632,-0.016647,0.034169,-0.032952,0.000000
04/01/2000,0.010043,0.012346,-0.012786,-0.044261,0.005277,-0.005666,0.000000,-0.018361,0.018019,0.013848,...,-0.014930,-0.025793,-0.009145,-0.031198,-0.019337,-0.015915,-0.007296,-0.047628,-0.011895,0.018868
05/01/2000,0.047628,-0.006192,0.011111,0.014493,-0.015915,0.000000,0.037955,-0.012089,-0.017858,0.036648,...,-0.002794,0.001810,0.010471,0.002866,-0.014799,-0.016645,0.037740,0.021506,0.000000,0.009479
06/01/2000,-0.011713,0.000000,0.032553,-0.027719,0.010695,0.005742,-0.003731,-0.003409,0.026907,0.013270,...,0.013341,-0.022196,0.011656,0.000000,0.004255,0.004829,0.052842,0.007926,-0.001714,0.009390
07/01/2000,-0.016118,0.003091,0.028573,0.033654,0.005249,0.003810,0.012975,0.000667,0.026202,0.007463,...,0.025864,0.027331,0.043898,0.006845,0.011073,0.035384,-0.011705,0.023408,-0.022551,0.009217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19/12/2003,0.008075,0.004848,-0.004301,-0.017213,-0.005277,0.001635,0.006424,0.004274,-0.004316,-0.000669,...,-0.006399,-0.007014,-0.006056,-0.003512,-0.006079,-0.001295,0.002782,-0.004605,-0.006079,0.004040
22/12/2003,0.011197,0.008683,0.000432,0.002708,0.001324,0.007368,0.003525,0.008505,-0.022748,0.001672,...,0.003384,0.004024,0.005592,0.000436,-0.003045,0.003559,0.003045,0.013751,0.020331,0.000000
23/12/2003,-0.016450,0.034613,0.008854,0.005367,0.000659,0.007338,0.000659,-0.005678,0.008842,0.005310,...,-0.004484,0.005493,0.004189,-0.002597,0.001016,0.001934,0.012658,0.028859,0.011313,0.002018
24/12/2003,-0.009945,-0.007892,0.007712,0.003117,0.006601,0.006509,-0.001318,-0.007117,0.010480,0.001330,...,-0.014926,-0.001994,-0.003257,-0.005610,0.002366,-0.001292,0.005286,0.010929,-0.002074,0.004032


In [24]:
import numpy as np

def EMA_WC(X, beta):
    """
    Calculate Exponential Moving Average Weighted Covariance matrix.

    Parameters:
        X (numpy.ndarray): Matrix of observations with shape (n, d) where n is the number of assets
                            and d is the number of days.
        beta (float): Exponential decay rate.

    Returns:
        numpy.ndarray: EMA-WC matrix.
    """
    _, d = X.shape

    weights = (1 - beta) * beta**np.arange(d - 1, -1, -1) / (1 - beta**d)

    return np.einsum('ti,tj->ij', X, X * weights)

def EMA_Weight(d, beta):
    """
    Calculate the W matrix as defined in the given equation.

    Parameters:
        d (int): Number of days.
        beta (float): Exponential decay rate.

    Returns:
        numpy.ndarray: W matrix.
    """
    return np.diag(d * (1 - beta) * beta**(np.arange(d)[::-1]) / (1 - beta**d))

def auxiliary_W(W, X):
    """
    Calculate the auxiliary observation matrix tilde(X) as defined in the given equation.

    Parameters:
        W (numpy.ndarray): Weight matrix.
        X (numpy.ndarray): Matrix of observations.

    Returns:
        numpy.ndarray: Auxiliary observation matrix tilde(X).
    """
    return np.sqrt(W) @ X


In [22]:
beta = 0.998
EMA = EMA_WC(X, beta)

In [21]:
EMA

array([[5.04e-06, 7.45e-07, 9.93e-07, ..., 1.45e-03, 1.07e-03, 3.15e-04],
       [7.38e-07, 4.18e-06, 4.84e-07, ..., 1.06e-03, 6.78e-04, 2.05e-04],
       [9.73e-07, 4.79e-07, 3.32e-06, ..., 4.83e-04, 1.00e-03, 2.23e-04],
       ...,
       [1.39e-06, 1.03e-06, 4.70e-07, ..., 1.39e-02, 9.12e-04, 4.68e-04],
       [1.01e-06, 6.47e-07, 9.66e-07, ..., 9.03e-04, 4.84e-03, 2.52e-04],
       [2.94e-07, 1.93e-07, 2.12e-07, ..., 4.59e-04, 2.49e-04, 1.23e-03]])